In [1]:
# Imports
import os
import time
import glob
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imutils.video import VideoStream
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import pandas as pd


In [2]:
# Constants
data_path = 'images'
mappings = {"r": "r", "p": "p", "s": "s"}
frame_size = (1080, 1920)
feature_names = [
    'area',
    'perimeter',
    'aspect_ratio',
    'extent',
    'convex_hull_area',           # Area of the convex hull
    'hull_perimeter',              # Perimeter of the convex hull
    'solidity',                    # Ratio of area to convex hull area
    'circularity',                 # Measure of how circular the shape is
    'eccentricity',                # Measure of the elongation of the shape
    'num_convexity_defects',       # Number of convexity defects
    'max_defect_depth',            # Maximum depth of convexity defects
    'major_axis_length',           # Length of the major axis of the fitted ellipse
    'minor_axis_length',           # Length of the minor axis of the fitted ellipse
    'orientation',                  # Orientation angle of the fitted ellipse
    'hu_moment',
    'centroid_x',
    'centroid_y'
]


categories = ['Rock', 'Paper', 'Scissor']
size = (320, 240)

lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Lower boundary of skin tone in HSV
upper_skin = np.array([20, 255, 255], dtype=np.uint8)  # Upper boundary of skin tone in HSV



In [3]:
# Initialize lists
data = np.empty((0, len(feature_names)), float)  # To store feature vectors
target = []  # To store labels
images = []  # List to store images
labels = []  # List to store corresponding labels

In [4]:
# Key mappings
kMappings = {ord(key): value for key, value in mappings.items()}

In [5]:
# Function definitions 
def getLargestContour(img_BW):
    """ Return largest contour in foreground as an nd.array """
    contours, hier = cv.findContours(img_BW.copy(), cv.RETR_EXTERNAL,
                                     cv.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv.contourArea)
    
    return np.squeeze(contour)

def getSimpleContourFeatures(contour):
    """ Return some simple contour features
        See https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_properties/py_contour_properties.html
    """       
    area = cv.contourArea(contour)
    perimeter = cv.arcLength(contour, True)
    x,y,w,h = cv.boundingRect(contour)
    aspect_ratio = float(w)/h
    rect_area = w*h
    extent = float(area)/rect_area
    features = np.array((area, perimeter, aspect_ratio, extent))
    
    return (features)

def unpackSimpleFeatures(data, target):
    # Check if data and target are correctly populated
    print(f"[INFO] Number of feature vectors: {data.shape[0]}")
    print(f"[INFO] Number of labels: {len(target)}")
    
    # Print the header (feature names)
    print(f"{'Image #':<10}{'Area':<15}{'Perimeter':<15}{'Aspect Ratio':<15}{'Extent':<15}{'Label'}")
    
    # Loop through each image's features and print them
    for i, feature_vector in enumerate(data):
        label = target[i]  # Get the label for this image
        
        area, perimeter, aspect_ratio, extent = feature_vector  # Unpack the features
        
        # Print the image index and its features
        print(f"{i:<10}{area:<15.2f}{perimeter:<15.2f}{aspect_ratio:<15.2f}{extent:<15.2f}{label}")


def getAdvancedContourFeatures(contour):
    """ Return advanced contour features based on contour analysis. """
    # Simple contour features
    area = cv.contourArea(contour)
    perimeter = cv.arcLength(contour, True)
    x, y, w, h = cv.boundingRect(contour)
    aspect_ratio = float(w) / h
    rect_area = w * h
    extent = float(area) / rect_area

    # Convex hull features
    hull = cv.convexHull(contour)
    hull_area = cv.contourArea(hull)
    hull_perimeter = cv.arcLength(hull, True)
    solidity = float(area) / hull_area if hull_area > 0 else 0

    # Circularity
    circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0

    # Eccentricity and fitting ellipse parameters
    try:
        ellipse = cv.fitEllipse(contour)
        (center, axes, orientation) = ellipse
        major_axis_length = max(axes)
        minor_axis_length = min(axes)
        eccentricity = np.sqrt(1 - (minor_axis_length / major_axis_length) ** 2) if major_axis_length > 0 else 0
    except:
        major_axis_length = minor_axis_length = orientation = eccentricity = 0

    # Convexity defects
    hull_indices = cv.convexHull(contour, returnPoints=False)
    defects = cv.convexityDefects(contour, hull_indices)
    num_convexity_defects = defects.shape[0] if defects is not None else 0
    max_defect_depth = max(defects[:, 0, 3]) / 256.0 if defects is not None else 0  # Normalize depth
    
    moments = cv.moments(contour)
    huMoments = cv.HuMoments(cv.moments(contour)).flatten()
    scaled_huMoments = -1.0 * np.sign(huMoments) * np.log10(abs(huMoments))
    
    # Centroid calculation
    centroid_x = int(moments['m10'] / moments['m00']) if moments['m00'] > 0 else 0
    centroid_y = int(moments['m01'] / moments['m00']) if moments['m00'] > 0 else 0
    centroid = (centroid_x, centroid_y)
    
    # Create a feature vector
    features = np.array([
        area,
        perimeter,
        aspect_ratio,
        extent,
        hull_area,
        hull_perimeter,
        solidity,
        circularity,
        eccentricity,
        num_convexity_defects,
        max_defect_depth,
        major_axis_length,
        minor_axis_length,
        orientation,
        scaled_huMoments[0],
        centroid_x,
        centroid_y
    ])
    
    return features

def unpackAdvancedFeatures(data, target):
    """Print the features and their corresponding labels."""
    # Check if data and target are correctly populated
    print(f"[INFO] Number of feature vectors: {data.shape[0]}")
    print(f"[INFO] Number of labels: {len(target)}")

    # Print the header (feature names)
    print(f"{'Image #':<10}{'Area':<15}{'Perimeter':<15}{'Aspect Ratio':<15}{'Extent':<15}{'Convex Hull Area':<20}{'Hull Perimeter':<20}{'Solidity':<15}{'Circularity':<15}{'Eccentricity':<15}{'Convexity Defects':<20}{'Max Defect Depth':<20}{'Major Axis Length':<20}{'Minor Axis Length':<20}{'Orientation':<15}{'Hu moments':<15}{'Centroid_x':<15}{'Centroid_y':<15}{'Label'}")

    # Loop through each image's features and print them
    for i, feature_vector in enumerate(data):
        label = target[i]  # Get the label for this image
        
        # Unpack the features
        (area, perimeter, aspect_ratio, extent,
         convex_hull_area, hull_perimeter, solidity, circularity,
         eccentricity, num_convexity_defects, max_defect_depth,
         major_axis_length, minor_axis_length, orientation, scaled_huMoments, centroid_x, centroid_y) = feature_vector
        
        # Print the image index and its features
        print(f"{i:<10}{area:<15.2f}{perimeter:<15.2f}{aspect_ratio:<15.2f}{extent:<15.2f}{convex_hull_area:<20.2f}{hull_perimeter:<20.2f}{solidity:<15.2f}{circularity:<15.2f}{eccentricity:<15.2f}{num_convexity_defects:<20}{max_defect_depth:<20.2f}{major_axis_length:<20.2f}{minor_axis_length:<20.2f}{orientation:<15.2f}{scaled_huMoments:<15.2f}{centroid_x:<15.2f}{centroid_y:<15.2f}{label}")


In [6]:
for category in categories:
    # Get the corresponding label from the mappings dictionary
    label = mappings[category[0].lower()]  # 'r' -> 0, 'p' -> 1, 's' -> 2
    
    # Form the path to the current category's images
    folder_path = os.path.join(data_path, category)
    
    # Get all image file paths in the current folder
    image_paths = glob.glob(os.path.join(folder_path, '*.jpg'))
    
    print(f"[INFO] Processing {category} images, found: {len(image_paths)}")
    
    # Loop over each image path
    for image_path in image_paths:
        # Load the image using OpenCV
        image = cv.imread(image_path)
        
        if image is None:
            print(f"[WARNING] Unable to load image {image_path}. Skipping.")
            continue
        
        # Resize the image to a standard frame size
        image_resized = cv.resize(image, frame_size)
        
        # Append the image and its label
        images.append(image_resized)  # Store the image
        labels.append(label)  # Store the corresponding label

# Convert images and labels to NumPy arrays for consistency
images = np.array(images)
labels = np.array(labels)

# Display some information about the dataset
print(f"[INFO] Number of images loaded: {len(images)}")
print(f"[INFO] Shape of images array: {images.shape}")
print(f"[INFO] Number of labels: {len(labels)}")

[INFO] Processing Rock images, found: 54
[INFO] Processing Paper images, found: 40
[INFO] Processing Scissor images, found: 39
[INFO] Number of images loaded: 133
[INFO] Shape of images array: (133, 1920, 1080, 3)
[INFO] Number of labels: 133


In [7]:
# Loop over each image and process for background removal
for i, image in enumerate(images):
    # Convert the image to the HSV color space
    img_hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

    # Create a binary mask where skin color is within the specified range
    mask = cv.inRange(img_hsv, lower_skin, upper_skin)

    # Apply morphological operations to clean up the mask (remove small noises)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    mask = cv.erode(mask, None, iterations=2)
    mask = cv.dilate(mask, None, iterations=2)

    # Create the result image by applying the mask to the original image
    result = cv.bitwise_and(image, image, mask=mask)

    # Convert the result to grayscale to proceed with feature extraction
    img_gray = cv.cvtColor(result, cv.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the grayscale image to reduce noise
    img_blurred = cv.GaussianBlur(img_gray, (5, 5), 0)

    # Apply binary thresholding to convert the image to binary (black & white)
    _, img_BW = cv.threshold(img_blurred, 110, 255, cv.THRESH_BINARY_INV)

    # Apply morphological operations to remove small noise and close gaps
    img_BW = cv.morphologyEx(img_BW, cv.MORPH_CLOSE, kernel)

    # Check if any foreground remains after thresholding
    if cv.countNonZero(img_BW) == 0:
        print(f"[WARNING] No foreground detected in image {i}. Skipping.")
        continue

    # Find the largest contour from the binary image
    try:
        contour = getLargestContour(img_BW)
    except ValueError:
        print(f"[WARNING] Could not find contours in image {i}. Skipping.")
        continue

    # Extract features from the largest contour
    features = getAdvancedContourFeatures(contour)
    
    # Append the features to the data matrix
    data = np.append(data, np.array([features]), axis=0)
    
    # Append the corresponding label from the 'labels' array
    target.append(labels[i])

    # Optionally, display the original image, mask, and result for visualization
    # cv.imshow("Original Image", image)
    # cv.imshow("Skin Mask", mask)
    # cv.imshow("Result after Background Removal", result)
    # cv.imshow("Binary Image for Contour", img_BW)

    # Wait for a key press and quit if 'q' is pressed
    k = cv.waitKey(1) & 0xFF
    if k == ord("q"):
        break

# Convert the target list to a NumPy array
target = np.array(target)

# Display the shape of the resulting feature dataset
print(f"[INFO] Feature extraction complete.")
print(f"[INFO] Shape of data matrix: {data.shape}")
print(f"[INFO] Number of target labels: {target.shape[0]}")

# Create a Bunch object containing the dataset
dataset = Bunch(data=data,
                target=target,
                feature_names=feature_names,
                unique_targets=np.unique(target))

print(f"[INFO] Dataset created with shape: {dataset.data.shape}")

[INFO] Feature extraction complete.
[INFO] Shape of data matrix: (133, 17)
[INFO] Number of target labels: 133
[INFO] Dataset created with shape: (133, 17)


In [8]:
unpackAdvancedFeatures(dataset.data, dataset.target)

[INFO] Number of feature vectors: 133
[INFO] Number of labels: 133
Image #   Area           Perimeter      Aspect Ratio   Extent         Convex Hull Area    Hull Perimeter      Solidity       Circularity    Eccentricity   Convexity Defects   Max Defect Depth    Major Axis Length   Minor Axis Length   Orientation    Hu moments     Centroid_x     Centroid_y     Label
0         1648058.50     8093.26        0.56           0.79           2070601.00          5996.00             0.80           0.32           0.73           1.0                 794.00              1079.50             732.90              168.13         0.56           500.00         890.00         r
1         1644666.50     8909.24        0.56           0.79           2070601.00          5996.00             0.79           0.26           0.37           1.0                 812.00              859.56              799.50              82.96          0.54           505.00         923.00         r
2         1609506.50     8828.36      

In [9]:
# Export to csv
"""Export the feature data and target labels to a CSV file."""
# Create a DataFrame from the feature data
f = pd.DataFrame(data, columns=feature_names)  # Use the feature_names list
# Ad the target labels as a new column
f['Label'] = target
# Expot to CSV
filename = 'data/hand_gesture_features.csv'
f.to_csv(filename, index=False)
print(f"[INFO] Data exported to {filename}")

[INFO] Data exported to data/hand_gesture_features.csv
